In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import os


# pre processamento

In [ ]:
df = pd.read_csv( r"D:\OneDrive\Área de Trabalho\saude\saude.csv", sep=";" , encoding="latin-1",low_memory=False, dtype="string")

In [ ]:
coluna_numerica = [  'Qtde Prescrita Farmácia Curitibana',
                     'Qtde Dispensada Farmácia Curitibana',
                     'Qtde de Medicamento Não Padronizado',
                     'Cômodos',
                     'cod_usuario',
                     'origem_usuario',
                     'residente',
                     'cod_profissional' ]

for col in coluna_numerica:
    df[col] = pd.to_numeric(  df[col].str.replace(',', '.')  )

for col in df.columns.tolist():
    if df[ col ].dtype == "string":
        df[col] = df[col].fillna("-")


df.loc[ df['Data do Internamento'] == '-', 'Data do Internamento']  = "01/01/2000 00:00:00"

In [ ]:
colunas_a_deconsiderar =  [  'Tipo de Unidade',
                             'Código da Unidade',
                             'Código do Procedimento',
                             'Código do CBO',
                             'Código do CID',
                             'Código do Tipo de Unidade'
                           ]       

df = df.drop( columns = colunas_a_deconsiderar )

In [ ]:
columns_enconder = [ 'Sexo',
                     'Descrição da Unidade',
                     'Descrição do Procedimento',
                     'Descrição do CBO',
                     'Descrição do CID',
                     'Solicitação de Exames',
                     'Encaminhamento para Atendimento Especialista',
                     'Área de Atuação',
                     'Desencadeou Internamento',                    
                     'Estabelecimento Solicitante',
                     'Estabelecimento Destino',
                     'CID do Internamento',
                     'Tratamento no Domicílio',
                     'Abastecimento',
                     'Energia Elétrica',
                     'Tipo de Habitação',
                     'Destino Lixo',
                     'Fezes/Urina',          
                     'Em Caso de Doença',
                     'Grupo Comunitário',
                     'Meio de Comunicacao',
                     'Meio de Transporte',
                     'Municício',
                     'Bairro',
                     'Nacionalidade',
                     ]

column_encoder = {}
for column in columns_enconder:
    column_encoder[ column ]  = LabelEncoder()
    column_encoder[ column ].fit( df[ column].unique().tolist() )
    df[ column ] = column_encoder[ column ].transform(df[ column ])

In [ ]:
df['Data do Atendimento'] = pd.to_datetime( df['Data do Atendimento'], format = "%d/%m/%Y %H:%M:%S" )
df['Data de Nascimento'] = pd.to_datetime( df['Data de Nascimento'] , format = "%d/%m/%Y %H:%M:%S")
df['Data do Internamento']  = pd.to_datetime( df['Data do Internamento']  , format = "%d/%m/%Y %H:%M:%S", errors="coerce")

df['Data do Atendimento'] = df['Data do Atendimento'].astype('int64')// 1e9
df['Data de Nascimento'] = df['Data de Nascimento'].astype('int64')// 1e9
df['Data do Internamento'] = pd.to_datetime(df['Data do Internamento'])
df['Data do Internamento'] = df['Data do Internamento'].astype('int64') // 1e9

df.loc[ df['Data do Internamento'] == 946684800.0, 'Data do Internamento' ] = 0.0

In [ ]:
colunas_selecionadas = [ 'Data do Atendimento','Descrição da Unidade', 'Data de Nascimento', 'Sexo', 'Meio de Transporte','Municício','Bairro','Descrição do CID']
df2= df[colunas_selecionadas]
df2.to_parquet(r"./base.parquet.gzip", compression="gzip")

# treinamento

In [2]:
df2 = pd.read_parquet( r"./base.parquet.gzip" )

In [13]:
data = df2.to_numpy(dtype=np.float32)

train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

del data

X_train = train_data[:, :-1]
y_train = train_data[:, -1]
X_test = test_data[:, :-1]
y_test = test_data[:, -1]

del train_data 
del test_data 

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

X_train_tensor = (X_train_tensor - X_train_tensor.mean()) / X_train_tensor.std()

input_size = X_train.shape[1]

del X_train, y_train, X_test, y_test

# Expandir os rótulos de destino para corresponder à forma da saída da rede
y_train_tensor_expanded = y_train_tensor.unsqueeze(1)
y_train_tensor_expanded = (y_train_tensor_expanded - y_train_tensor_expanded.mean()) / y_train_tensor_expanded.std()



In [59]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True,
        )
        self.fc  = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, self.hidden_size)
        out, hidden = self.rnn(x, h0)
        
        return out, hidden


In [ ]:
hidden_size = 15
output_size = 1
learning_rate = 0.01
num_epochs = 100
batch_size = 5012 
num_samples = X_train_tensor.size(0)


print(input_size) 
model = Net(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
writer = SummaryWriter('runs/rnn_experiment')

hidden_prev = torch.zeros( 1, hidden_size )

if os.path.exists( r"model.pth" ):
    print("lendo modelo")
    model.load_state_dict(torch.load('model.pth'))

# Treinar o modelo
for epoch in range(num_epochs):
    epoch_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    for i in range(0, num_samples, batch_size):
        X_batch = X_train_tensor[i:i+batch_size]
        y_batch = y_train_tensor_expanded[i:i+batch_size]       

        outputs, hidden_prev = model(X_batch)
        hidden_prev = hidden_prev.detach()
        
        loss = criterion(outputs.squeeze(), y_batch)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_item = loss.item()
        epoch_loss += loss_item

        _, predicted = torch.max(outputs.data, 1)
        total_samples += y_batch.size(0)
        correct_predictions += (predicted == y_batch).sum().item()
        
    
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_item:.4f}', end="\r")

        accuracy = 100 * correct_predictions / total_samples
        epoch_loss = epoch_loss / (num_samples / batch_size)
        torch.save( model.state_dict(), 'model.pth' )
        writer.add_scalar('Loss/train', epoch_loss, epoch)
        writer.add_scalar('Accuracy/train', accuracy, epoch)


    